In [2]:
# =============================================================================
#
# FINAL SCRIPT: Crypto Price Prediction
#
# =============================================================================

# --- 1. SETUP: Import Libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Install pandas-ta if needed (run once, then comment out)
# !pip install git+https://github.com/tradingview/pandas-ta.git
import pandas_ta as ta

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


# --- 2. DATA LOADING & FEATURE ENGINEERING ---

def load_and_clean_coin(ticker_symbol):
    """Loads and cleans the data for a given crypto ticker."""
    file_path = os.path.join('data', f'{ticker_symbol}_data.csv')
    col_names = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    try:
        df = pd.read_csv(
            file_path, header=None, skiprows=3, names=col_names,
            index_col='Date', parse_dates=True
        )
        df.dropna(inplace=True)
        return df
    except Exception as e:
        return None

def create_advanced_features(df):
    """Creates a comprehensive set of features for modeling."""
    if df is None: return None
    df_featured = df.copy()
    for i in range(1, 8):
        df_featured[f'Close_lag_{i}'] = df_featured['Close'].shift(i)
    df_featured['MA_7'] = df_featured['Close'].rolling(window=7).mean()
    df_featured['MA_30'] = df_featured['Close'].rolling(window=30).mean()
    df_featured['Volatility_7'] = df_featured['Close'].rolling(window=7).std()
    df_featured['RSI_14'] = ta.rsi(df_featured['Close'], length=14)
    df_featured['Target'] = df_featured['Close'].shift(-1)
    return df_featured


# --- 3. MODEL TRAINING & EVALUATION ---

def train_and_evaluate(model, df, ticker_symbol):
    """A reusable function to train and evaluate any given model."""
    model_name = model.__class__.__name__
    feature_cols = [col for col in df.columns if 'lag' in col or 'MA' in col or 'Volatility' in col or 'RSI' in col]
    model_df = df[feature_cols + ['Target']].copy()
    model_df.dropna(inplace=True)
    
    X = model_df[feature_cols]
    y = model_df['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    print(f"✅ {model_name} Trained. MAE: ${mae:,.2f}")
    return model, mae


# --- 4. PREDICTION FUNCTION ---

def predict_tomorrow(model, df):
    """
    Uses the trained model to predict the next day's price.
    """
    # Get the most recent row of data that has all features calculated
    latest_data = df.dropna().iloc[-1]
    
    # Prepare the feature set for prediction
    feature_cols = [col for col in df.columns if 'lag' in col or 'MA' in col or 'Volatility' in col or 'RSI' in col]
    prediction_input = latest_data[feature_cols].values.reshape(1, -1)
    
    # Make the prediction
    predicted_price = model.predict(prediction_input)
    
    return predicted_price[0]


# --- 5. MAIN EXECUTION ---

# Load and prepare data
btc_df = load_and_clean_coin('BTC-USD')
btc_featured_df = create_advanced_features(btc_df)

if btc_featured_df is not None:
    # Initialize models
    linear_model = LinearRegression()
    
    # Train and evaluate Linear Regression
    champion_model, champion_mae = train_and_evaluate(linear_model, btc_featured_df, 'BTC-USD')

    print("\n--- DEPLOYING CHAMPION MODEL ---")
    # Use the champion model to make a prediction
    predicted_price_tomorrow = predict_tomorrow(champion_model, btc_featured_df)

    print(f"\n📈 The model predicts that the closing price for BTC tomorrow will be: ${predicted_price_tomorrow:,.2f}")

✅ LinearRegression Trained. MAE: $1,330.59

--- DEPLOYING CHAMPION MODEL ---

📈 The model predicts that the closing price for BTC tomorrow will be: $112,578.49


/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
